# HandsOn Week 13
Welcome to handsOn week 13: graph in distributed system, using GraphFrames. To use GraphFrames, you need to run ```pyspark --packages graphframes:graphframes:0.8.0-spark2.4-s_2.11``` (adjust with your Spark version. That script uses Spark 2.4.x, the one used in our VM, and it will download the graphframes package automatically, thus, make sure you have internet connection when launching the script).

Note: you can use any Spark & GraphFrames API (without building from-the-scratch).

### Read The Dataset and Build The Graph

In [ ]:
from pyspark.sql.types import *
from graphframes import *

def create_transport_graph(nodes_filePath, edges_filePath):
    node_fields = [
        StructField("id", StringType(), True),
        StructField("latitude", FloatType(), True),
        StructField("longitude", FloatType(), True),
        StructField("population", IntegerType(), True)
    ]
    nodes = spark.read.csv(nodes_filePath, header=True,
                           schema=StructType(node_fields))
    edge_fields = [
        StructField("src", StringType(), True),
        StructField("dst", StringType(), True),
        StructField("relationship", StringType(), True),
        StructField("cost", FloatType(), True)
    ]
    rels = spark.read.csv(edges_filePath, header=True,
                         schema=StructType(edge_fields))
    reversed_rels = (rels.withColumn("newSrc", rels.dst)
                     .withColumn("newDst", rels.src)
                     .drop("dst", "src")
                     .withColumnRenamed("newSrc", "src")
                     .withColumnRenamed("newDst", "dst")
                     .select("src", "dst", "relationship", "cost"))

    relationships = rels.union(reversed_rels)
    
    return GraphFrame(nodes, relationships)

g = create_transport_graph("./transport-nodes.csv", "./transport-relationships.csv")
g.vertices.show()
g.edges.show()

+----------------+---------+---------+----------+
|              id| latitude|longitude|population|
+----------------+---------+---------+----------+
|       Amsterdam| 52.37919| 4.899431|    821752|
|         Utrecht|52.092876|  5.10448|    334176|
|        Den Haag|52.078663| 4.288788|    514861|
|       Immingham| 53.61239| -0.22219|      9642|
|       Doncaster| 53.52285| -1.13116|    302400|
|Hoek van Holland|  51.9775|  4.13333|      9382|
|      Felixstowe| 51.96375|   1.3511|     23689|
|         Ipswich| 52.05917|  1.15545|    133384|
|      Colchester| 51.88921|  0.90421|    104390|
|          London|51.509865|-0.118092|   8787892|
|       Rotterdam|  51.9225|  4.47917|    623652|
|           Gouda| 52.01667|  4.70833|     70939|
+----------------+---------+---------+----------+

+----------------+----------------+------------+-----+
|             src|             dst|relationship| cost|
+----------------+----------------+------------+-----+
|       Amsterdam|         Utrecht

## Milestone 01
1. Find source-destination airport pair/pairs with maximum cost
2. Find source-destination airport pair/pairs with minimum cost
3. Calculate the average cost

In [ ]:
# write your code here
maximum_cost = g.edges.groupBy().max("cost")
maximum_cost_pairs = g.edges.filter(f"cost = {maximum_cost.collect()[0][0]}")
maximum_cost_pairs.show()

minimum_cost = g.edges.groupBy().min("cost")
minimum_cost_pairs = g.edges.filter(f"cost = {minimum_cost.collect()[0][0]}")
minimum_cost_pairs.show()

average_cost = g.edges.groupBy().avg("cost")
average_cost.show()

<img title="Milestone 1" align="left" src="milestone1.png" alt="Drawing" style="width: 600px;"/>

### Milestone 02
1. Find flight routes that have no direct connection

In [ ]:
# write your code here
flight_routes = g.find("(a)-[]->(b);(b)-[]->(c);!(a)-[]->(c)").filter("a.id != c.id")
flight_routes.show()

<img title="Milestone 2" align="left" src="milestone2.png" alt="Drawing" style="width: 600px;"/>

### Milestone 03
1. Find the most important airport (you can use any measurement to judge the importance level of airports, and please describe why you choose it -the measurement-)

In [ ]:
# write your code here
degree_centrality = g.degrees
most_important_airport = g.vertices.filter(f"id = '{degree_centrality.orderBy(degree_centrality.degree.desc()).collect()[0][0]}'")
most_important_airport.show()

# Measurement yang dipilih adalah derajat sentralitas, yang menghitung jumlah koneksi langsung (penerbangan masuk dan keluar) yang dimiliki suatu bandara. 
# Bandara dengan derajat sentralitas yang lebih tinggi dianggap lebih penting karena memiliki lebih banyak rute langsung ke bandara lain, yang menunjukkan bahwa bandara tersebut merupakan hub utama dalam jaringan.

<img title="Milestone 3" align="left" src="milestone3.png" alt="Drawing" style="width: 600px;"/>

### Milestone 04
1. Find the sortest path based on "node" -path with fewest nodes- from Amsterdam to London 

In [ ]:
# write your code here
shortest_path = g.bfs(
    fromExpr="id = 'Amsterdam'",
    toExpr=f"id = 'London'"
)
shortest_path.show()

<img title="Milestone 4" align="left" src="milestone4.png" alt="Drawing" style="width: 600px;"/>

### Bonus
1. Find the sortest path based on "cost value" -see 'cost' column in the edge/relationship dataframe - from Amsterdam to London 

In [ ]:
# write your code here
import networkx as nx

G_nx = nx.Graph()
for row in g.edges.collect():
    G_nx.add_edge(row.src, row.dst, weight=row.cost)
shortest_path = nx.shortest_path(G_nx, source='Amsterdam', target='London', weight='weight')
shortest_path

<img title="Milestone 5" align="left" src="milestone5.png" alt="Drawing" style="width: 600px;"/>

# Submission
Submit this ```ipynb``` file to the course portal, with format: ```HandsOnWeek13_NIM_NamaLengkap.ipynb```. Make sure when submitting this file, each code cell has the outputs (not blank).